In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch, json, re, os

os.environ["TORCH_COMPILE_DISABLE"] = "1"   # PyTorch 2.8에서 안전
torch.set_grad_enabled(False)

MODEL_ID = "openai/gpt-oss-20b"

tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,   # gpt-oss 통합(MXFP4) 경로 활성
    low_cpu_mem_usage=True,
)

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
)

# 간단 구동 체크
resp = gen([{"role": "user", "content": "안녕! 테스트야."}], max_new_tokens=40, do_sample=False)
print(resp[0]["generated_text"][-1]["content"])


c:\Users\Admin\anaconda3\envs\gptoss\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.65s/it]
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


analysisThe user says "안녕! 테스트야." which is Korean: "Hello! This is a test." They likely want a friendly response. The instruction: "You are ChatGPT


In [5]:
import os, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 안전: 토치 컴파일 끄기
os.environ["TORCH_COMPILE_DISABLE"] = "1"

MODEL_ID = "openai/gpt-oss-20b"

tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map=None,           # <- auto 금지
    low_cpu_mem_usage=False,   # <- 지연 로드 금지 (meta 방지)
)
model.to("cuda")               # <- 즉시 materialize
model.eval()

gen = pipeline("text-generation", model=model, tokenizer=tok)

# 짧게 구동 체크
out = gen([{"role": "user", "content": "안녕! 테스트야."}],
          max_new_tokens=40, do_sample=False)[0]["generated_text"]
print(out[-1]["content"])


You have loaded an FP4 model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model. To remove this warning, pass device_map = 'cuda'. 
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.10s/it]
Device set to use cuda:0


analysisThe user says "안녕! 테스트야." which is Korean: "Hello! This is a test." They likely want a friendly response. The instruction: "You are ChatGPT


In [6]:
import json, re

SYSTEM = (
    "You are EmotionScore. Respond with JSON ONLY. "
    "Return exactly these four keys as floats in [0,1]: "
    "dopamine, serotonin, norepinephrine, melatonin. "
    "Do not write any other text, no explanation."
)

def emotion_vector(text: str):
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"Text: {text}"},
        {"role": "assistant", "content": "{"}  # JSON 강제 시작
    ]
    out = gen(messages, max_new_tokens=60, do_sample=False,
              return_full_text=False)[0]["generated_text"]
    content = "{" + out

    m = re.search(r"\{.*\}", content, re.S)
    if not m:
        raise RuntimeError(f"JSON not found in: {content[:200]}")
    data = json.loads(m.group())

    def c(x): 
        try: return max(0.0, min(1.0, float(x)))
        except: return 0.0
    vec = [c(data.get(k)) for k in ("dopamine","serotonin","norepinephrine","melatonin")]
    return vec, data

# 테스트
demo = "우울하다"
vec, raw = emotion_vector(demo)
print("emotion vector [DA,5-HT,NE,MLT] =", vec)
print("raw =", raw)


emotion vector [DA,5-HT,NE,MLT] = [0.2, 0.3, 0.4, 0.6]
raw = {'dopamine': 0.2, 'serotonin': 0.3, 'norepinephrine': 0.4, 'melatonin': 0.6}


In [7]:
def fetch_scores(text: str):
    vec, raw = emotion_vector(text)  # 여기서 vec=[DA,HT,NE,MLT], raw는 JSON
    return vec, raw

fetch_scores("오늘 너무 행복해!")

([0.92, 0.88, 0.75, 0.3],
 {'dopamine': 0.92,
  'serotonin': 0.88,
  'norepinephrine': 0.75,
  'melatonin': 0.3})

**흥분성 뉴런**

In [8]:
from dataclasses import dataclass

@dataclass
class ModNeuronCfg:
    v_leak: float = 0.90            # 기억이 없어지는 정도(누설 전위)
    g0: float = 1.0                 # 입력 신호의 기본 증폭률
    a_ne: float = 0.8               # NE의 증폭률
    a_mlt: float = 1.0              # MLT의 억제률
    theta0: float = 1.0             # 초기 임계 값
    k_theta: float = 0.6            # 세로토닌의 임계값 증가율
    theta_homeo: float = 0.2        # 스파이크가 발생하면 임계를 조금 낮추는 값(발화 후 다시 발화할 수 있게 임계 살짝 내려감)
    theta_min: float = 0.2          # 임계값 최소
    theta_max: float = 2.5          # 임계값 최대
    W0: float = 0.5                 # 초기 가중치
    eta: float = 0.25               # 학습률
    rho: float = 0.9                # 적격도 감쇠율
    # 적격도: 시냅스가 최근에 활성된 사실을 잠시 기억해 두었다가, 나중에 보상 신호(도파민 등)가 도착했을 때 그 연결을 강화하거나 약화할 수 있게 해 주는 일시적 흔적
    W_min: float = 0.0              # 가중치 최소
    W_max: float = 3.0              # 가중치 최대
    v_min: float = -1.0             # 전위 최소
    v_max: float = 5.0              # 전위 최대


In [9]:
class MiniNeuron:
    def __init__(self, fetch_scores, cfg: ModNeuronCfg = ModNeuronCfg()):
        self.cfg = cfg
        self.fetch_scores = fetch_scores
        self.v = 0.0 # 뉴런의 막 전위

    def step(self, text: str, x_t: float = 1.0):
        (DA, HT, NE, MLT), raw = self.fetch_scores(text)  # vec = [DA, HT, NE, MLT]
        g = self.cfg.g0 * (1.0 + self.cfg.a_ne * NE - self.cfg.a_mlt * MLT)  # gain(g): 입력 신호를 얼마나 크게 증폭/감쇠할지 결정하는 계수
        g = max(0.0, g)  # 음수 방지
        I = g * x_t # 받은 입력 자극
        self.v = self.cfg.v_leak * self.v + I
        return {"mods":{"DA":DA,"HT":HT,"NE":NE,"MLT":MLT},
                "gain": g, "I": I, "v": self.v, "raw": raw}
    
mn = MiniNeuron(fetch_scores)

In [10]:
o1 = mn.step("긴장감이 올라 집중이 잘 된다. 빠르게 대응한다.")
print("NE↑ :", {k:round(v,3) if isinstance(v,float) else v for k,v in o1.items() if k in ("gain","I","v")})

NE↑ : {'gain': 1.504, 'I': 1.504, 'v': 1.504}


In [11]:
o2 = mn.step("피곤하고 졸리다. 의욕이 떨어진다.")  # MLT↑ 기대
print("MLT↑:", {k:round(v,3) if isinstance(v,float) else v for k,v in o2.items() if k in ("gain","I","v")})

MLT↑: {'gain': 0.28, 'I': 0.28, 'v': 1.634}


In [12]:
#mini neuron에서 업그레이드 버전
class SpikeNeuron:
    def __init__(self, fetch_scores, cfg: ModNeuronCfg = ModNeuronCfg()):
        self.cfg = cfg
        self.fetch_scores = fetch_scores
        self.v = 0.0
        self.theta = cfg.theta0
        self.W = cfg.W0

    def step(self, text: str, x_t: float = 1.0):
        (DA, HT, NE, MLT), raw = self.fetch_scores(text)

        dW = 0.0  # 가중치 변화량 초기화

        # 입력 게인 계산
        g = self.cfg.g0 * (1.0 + self.cfg.a_ne * NE - self.cfg.a_mlt * MLT)
        g = max(0.0, g)

        # 막전위 계산
        I = g * x_t
        self.v = self.cfg.v_leak * self.v + I

        # 스파이크 여부 결정
        spike = 1 if self.v >= self.theta else 0
        if spike:
            self.v = 0.0  # 발화 후 막전위 리셋
            # 도파민이 클 수록 가중치 증가
            dW = self.cfg.eta * DA
            self.W = min(self.cfg.W_max, max(self.cfg.W_min, self.W + dW))

        return {
            "mods": {"NE": NE, "MLT": MLT},
            "gain": g, "I": I, "v": self.v,
            "theta": self.theta, "spike": spike,
            "W": self.W, "dW": dW
        }
sn = SpikeNeuron(fetch_scores)

In [13]:
for i in range(6):
    out = sn.step("긴장감이 올라 집중이 잘 된다. 빠르게 대응한다.", x_t=0.2)  # NE↑ 문장 x_t: 입력 세기
    print(f"step {i}: v={out['v']:.3f}, theta={out['theta']:.3f}, spike={out['spike']}")


step 0: v=0.301, theta=1.000, spike=0
step 1: v=0.572, theta=1.000, spike=0
step 2: v=0.815, theta=1.000, spike=0
step 3: v=0.000, theta=1.000, spike=1


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


step 4: v=0.301, theta=1.000, spike=0
step 5: v=0.572, theta=1.000, spike=0


**억제성 뉴런**

In [14]:
from dataclasses import dataclass, asdict

@dataclass
class InhibNeuronCfg:
    v_leak: float = 0.9
    theta0: float = 1.0
    W0: float = 0.5
    W_min: float = 0.0
    W_max: float = 3.0
    eta: float = 0.2  # 학습률 기본치

In [15]:
class InhibitoryNeuron:
    """
    억제성 뉴런 (GABA):
      - 입력 적분 → 발화(spike) → downstream 뉴런 억제 신호
      - 세로토닌 HT: 억제성 강화/약화 비율에 큰 영향
      - 도파민 DA: 보상 학습 비율에 영향
    """
    def __init__(self, fetch_scores, cfg: InhibNeuronCfg = InhibNeuronCfg()):
        self.cfg = cfg
        self.fetch_scores = fetch_scores
        self.v = 0.0
        self.theta = cfg.theta0
        self.W = cfg.W0

    def step(self, text: str, x_t: float = 1.0):
        (DA, HT, NE, MLT), raw = self.fetch_scores(text)

        dW = 0.0  # 가중치 변화량 초기화

        # 막전위 계산
        I = self.W * x_t
        self.v = self.cfg.v_leak * self.v + I

        # 스파이크 여부
        spike = 1 if self.v >= self.theta else 0
        if spike:
            self.v = 0.0  # 발화 후 리셋

        # 학습 규칙 (세로토닌, 도파민에 따른 가중치 업데이트)
        if spike:
            # 강화 비율
            dW_up = (0.9 * HT) + (0.7 * DA)
            # 약화 비율
            dW_down = (0.1 * HT) + (0.3 * DA)

            # 변화량 = eta * (강화 - 약화)
            dW = self.cfg.eta * (dW_up - dW_down)

            # 가중치 업데이트
            self.W = min(self.cfg.W_max, max(self.cfg.W_min, self.W + dW))

        return {
            "mods": {"DA": DA, "HT": HT, "NE": NE, "MLT": MLT},
            "I": I, "v": self.v, "theta": self.theta, "spike": spike,
            "W": self.W, "dW": dW, "raw": raw
        }
    
inh = InhibitoryNeuron(fetch_scores)

In [16]:
# 세로토닌 강화 케이스
for i in range(5):
    out = inh.step("마음이 차분하고 안정적이다. 욕구가 억제된다.")
    print(i, out["v"], out["spike"], out["W"])


0 0.5 0 0.5
1 0.95 0 0.5
2 0.0 1 0.652
3 0.652 0 0.652
4 0.0 1 0.804


In [17]:
text = "드디어 대회에 합격했다! 팀원들과 소리 지르며 기뻐했다."

for i in range(5):
    out = inh.step(text, x_t=0.2)
    print(f"{i} v={out['v']:.3f} spike={out['spike']} W={out['W']:.3f} dW={out['dW']:.4f}")

0 v=0.161 spike=0 W=0.804 dW=0.0000
1 v=0.306 spike=0 W=0.804 dW=0.0000
2 v=0.436 spike=0 W=0.804 dW=0.0000
3 v=0.553 spike=0 W=0.804 dW=0.0000
4 v=0.658 spike=0 W=0.804 dW=0.0000


In [18]:
from dataclasses import dataclass

@dataclass
class ModulatoryNeuronCfg:
    eta_base: float = 0.2      # 기본 학습률
    theta_base: float = 1.0    # 기본 임계값
    gain_base: float = 1.0     # 기본 게인

class ModulatoryNeuron:
    """
    조절성 뉴런:
      - 직접 발화하지 않고, 다른 뉴런의 파라미터(θ, η, W 등)를 조정
    """
    def __init__(self, fetch_scores, cfg: ModulatoryNeuronCfg = ModulatoryNeuronCfg()):
        self.cfg = cfg
        self.fetch_scores = fetch_scores

    def modulate(self, text: str, neuron_params: dict):
        (DA, HT, NE, MLT), raw = self.fetch_scores(text)

        # 조절
        theta = neuron_params.get("theta", self.cfg.theta_base)
        eta   = neuron_params.get("eta", self.cfg.eta_base)
        gain  = neuron_params.get("gain", self.cfg.gain_base)

        # 도파민: 학습률 증가 (동기/보상)
        eta *= (1.0 + 0.5 * DA)

        # 세로토닌: 임계값 증가 (안정화)
        theta *= (1.0 + 0.5 * HT)

        # 노르아드레날린: 전체 게인 증가(각성)
        gain *= (1.0 + 0.8 * NE)

        # 아세틸콜린: 발화율 높은 상황에서 학습률 추가 조정 (여기선 단순히 +20%)
        if neuron_params.get("firing_rate", 0) > 0.5:
            eta *= 1.2

        return {"theta": theta, "eta": eta, "gain": gain,
                "mods": {"DA": DA, "HT": HT, "NE": NE, "MLT": MLT}, "raw": raw}
mod = ModulatoryNeuron(fetch_scores)


In [19]:
# 어떤 뉴런이 현재 가진 파라미터 예시
neuron_params = {"theta": 1.0, "eta": 0.2, "gain": 1.0, "firing_rate": 0.6}

out = mod.modulate("긴장되고 흥분되며 보상이 기대된다.", neuron_params)
print(out)

{'theta': 1.275, 'eta': 0.342, 'gain': 1.6400000000000001, 'mods': {'DA': 0.85, 'HT': 0.55, 'NE': 0.8, 'MLT': 0.1}, 'raw': {'dopamine': 0.85, 'serotonin': 0.55, 'norepinephrine': 0.8, 'melatonin': 0.1}}


In [20]:
# 테스트 문장 (강한 보상, 각성)
text = "드디어 대회에 합격했다! 팀원들과 소리 지르며 기뻐했다."

# 조절성 뉴런으로 파라미터 조정
mod_out = mod.modulate(
    text,
    {"theta": sn.theta, "eta": sn.cfg.eta, "gain": sn.cfg.g0, "firing_rate": 0.6}
)
print("조절성 출력:", mod_out)

# SpikeNeuron에 반영
sn.theta = mod_out["theta"]        # 임계값 조정
sn.cfg.eta = mod_out["eta"]        # 학습률 업데이트
sn.cfg.g0 = mod_out["gain"]        # 게인 업데이트

# 여러 step 실행
for i in range(8):
    out = sn.step(text, x_t=0.3)
    print(f"step {i}: v={out['v']:.3f}, theta={out['theta']:.3f}, "
          f"gain={out['gain']:.3f}, spike={out['spike']}, dW={out['dW']:.3f}")

조절성 출력: {'theta': 1.425, 'eta': 0.438, 'gain': 1.624, 'mods': {'DA': 0.92, 'HT': 0.85, 'NE': 0.78, 'MLT': 0.12}, 'raw': {'dopamine': 0.92, 'serotonin': 0.85, 'norepinephrine': 0.78, 'melatonin': 0.12}}
step 0: v=1.247, theta=1.425, gain=2.442, spike=0, dW=0.000
step 1: v=0.000, theta=1.425, gain=2.442, spike=1, dW=0.403
step 2: v=0.733, theta=1.425, gain=2.442, spike=0, dW=0.000
step 3: v=1.392, theta=1.425, gain=2.442, spike=0, dW=0.000
step 4: v=0.000, theta=1.425, gain=2.442, spike=1, dW=0.403
step 5: v=0.733, theta=1.425, gain=2.442, spike=0, dW=0.000
step 6: v=1.392, theta=1.425, gain=2.442, spike=0, dW=0.000
step 7: v=0.000, theta=1.425, gain=2.442, spike=1, dW=0.403


In [21]:
import numpy as np

class Network:
    def __init__(self, fetch_scores, n_exc=3, n_inh=2, n_mod=1):
        self.exc_neurons = [SpikeNeuron(fetch_scores) for _ in range(n_exc)]
        self.inh_neurons = [InhibitoryNeuron(fetch_scores) for _ in range(n_inh)]
        self.mod_neurons = [ModulatoryNeuron(fetch_scores) for _ in range(n_mod)]

        # 뉴런 전체 리스트 (인덱스로 관리)
        self.neurons = self.exc_neurons + self.inh_neurons + self.mod_neurons
        self.n_total = len(self.neurons)

        # 연결 가중치 행렬 초기화
        self.connections = np.random.uniform(0.1, 1.0, size=(self.n_total, self.n_total))

        # 억제성 뉴런 인덱스
        self.exc_idx = range(0, n_exc)
        self.inh_idx = range(n_exc, n_exc + n_inh)
        self.mod_idx = range(n_exc + n_inh, self.n_total)

        # 자기 연결 제거
        np.fill_diagonal(self.connections, 0.0)

        # 억제 뉴런 → 모든 출력은 음수로
        for i in self.inh_idx:
            self.connections[i, :] = -np.abs(self.connections[i, :])

        # 행 정규화 (폭주 방지)
        row_norm = np.maximum(np.sum(np.abs(self.connections), axis=1, keepdims=True), 1e-6)
        self.connections = self.connections / row_norm * 0.5  # 각 행 합 ≈ 0.5

        # 재귀 입력 혼합 비율
        self.alpha = 0.3  

    def step(self, text: str, inputs=None):
        if inputs is None:
            # 외부 기본 입력 (약한 값)
            inputs = np.array([0.1] * self.n_total)
        else:
            inputs = np.array(inputs)

        spikes = [0] * self.n_total
        outputs = []

        # 각 뉴런 업데이트
        for i, neuron in enumerate(self.neurons):
            if isinstance(neuron, ModulatoryNeuron):
                # 조절성 뉴런은 파라미터 조정
                mod_out = neuron.modulate(text, {
                    "theta": self.neurons[0].theta,
                    "eta": self.neurons[0].cfg.eta,
                    "gain": self.neurons[0].cfg.g0,
                    "firing_rate": 0.5,
                })
                # 흥분성 뉴런 파라미터에 적용
                for exc in self.exc_neurons:
                    exc.theta = mod_out["theta"]
                    exc.cfg.eta = mod_out["eta"]
                    exc.cfg.g0 = mod_out["gain"]
                out = mod_out
                spike = 0
            else:
                out = neuron.step(text, x_t=inputs[i])
                spike = out["spike"]

            spikes[i] = spike
            outputs.append(out)

        spikes = np.array(spikes)

        # 스파이크 결과를 다음 step 입력에 반영
        inputs_next = np.dot(spikes, self.connections)

        # 포화/클리핑 적용
        inputs_next = np.clip(inputs_next, 0, 1)

        # 외부 입력과 혼합 (alpha 비율)
        inputs_mixed = self.alpha * inputs_next + (1 - self.alpha) * inputs

        return {
            "spikes": spikes.tolist(),
            "outputs": outputs,
            "inputs_next": inputs_mixed.tolist()
        }
net = Network(fetch_scores, n_exc=3, n_inh=2, n_mod=1)

In [22]:
text = "드디어 대회에 합격했다! 팀원들과 소리 지르며 기뻐했다."
for t in range(5):
    result = net.step(text)
    print(f"Step {t}: spikes={result['spikes']}, inputs_next={result['inputs_next']}")

Step 0: spikes=[0, 0, 0, 0, 0, 0], inputs_next=[0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999]
Step 1: spikes=[0, 0, 0, 0, 0, 0], inputs_next=[0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999]
Step 2: spikes=[0, 0, 0, 0, 0, 0], inputs_next=[0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999]
Step 3: spikes=[0, 0, 0, 0, 0, 0], inputs_next=[0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999]
Step 4: spikes=[0, 0, 0, 0, 0, 0], inputs_next=[0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999, 0.06999999999999999]
